<a href="https://colab.research.google.com/github/ykitaguchi77/LangGraph/blob/main/CRAG_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**LangGraph CRAG**

https://github.com/langchain-ai/langgraph/blob/main/examples/rag/langgraph_crag.ipynb?ref=blog.langchain.dev

In [ ]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain langgraph tavily-python --q

In [17]:
# APIの設定
from google.colab import drive
drive.mount("/content/drive")

with open("/content/drive/MyDrive/Deep_learning/api.txt") as file:
    #text = file.read()
    i=1
    key = []
    while True:
        include_break_line = file.readline() #改行が含まれた行
        line = include_break_line.rstrip() #改行を取り除く
        if line: #keyの読み込み
            #print(f'{i}行目：{line}')
            key.append(line)
            i += 1
        else:
            break

# APIキーの準備
# #ngrok_aceess_token = key[5]
#openai_api_key = key[3]
# deepl_auth_key = key[1]
# serp_api_key = key[7]

import os
os.environ["OPENAI_API_KEY"] = key[3]
os.environ["SERPAPI_API_KEY"] = key[7]
os.environ["GOOGLE_CSE_ID"] = key[9]
os.environ["GOOGLE_API_KEY"] = key[11]
os.environ["ANTHROPIC_API_KEY"] = key[21]
os.environ["TAVILY_API_KEY"] = key[25]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#**Index**

In [18]:
# 必要なライブラリをインポート
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# 処理対象のURLを定義
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

# 各URLからコンテンツをロード
docs = [WebBaseLoader(url).load() for url in urls]
# ロードした文書を1つのリストにフラット化
docs_list = [item for sublist in docs for item in sublist]

# テキスト分割器を設定
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
# 文書を小さなチャンクに分割
doc_splits = text_splitter.split_documents(docs_list)

# ベクトルデータベースを作成し、分割した文書を追加
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=OpenAIEmbeddings(),
)
# 検索機能（retriever）を設定
retriever = vectorstore.as_retriever()

In [19]:
### 検索結果評価器

# 必要なライブラリをインポート
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI


# データモデルの定義
class GradeDocuments(BaseModel):
    """検索された文書の関連性チェックのためのバイナリスコア"""

    binary_score: str = Field(
        description="文書が質問に関連しているかどうか、'yes' または 'no'"
    )


# 関数呼び出し機能付きLLMの設定
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# プロンプトの設定
system = """あなたは、検索された文書がユーザーの質問に関連しているかを評価する採点者です。\n
    文書に質問に関連するキーワードや意味的な内容が含まれている場合、それを関連あるものとして評価してください。\n
    文書が質問に関連しているかどうかを示すために、'yes' または 'no' のバイナリスコアを与えてください。"""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "検索された文書: \n\n {document} \n\n ユーザーの質問: {question}"),
    ]
)

# 評価器の作成
retrieval_grader = grade_prompt | structured_llm_grader

# テスト用の質問と文書の準備
question = "agent memory"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content

# 評価の実行と結果の表示
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='yes'


In [20]:
### 生成（日本語版）

# 必要なライブラリをインポート
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 日本語用のRAGプロンプトを定義
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは親切で優秀な日本語のアシスタントです。与えられたコンテキストを使用して、ユーザーの質問に日本語で回答してください。"),
    ("human", "コンテキスト：\n{context}\n\n質問：{question}\n\n回答：")
])

# LLMの設定
# OpenAIのGPT-3.5-turboモデルを使用し、temperature=0で決定論的な出力を得る
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# 後処理関数の定義
# 複数の文書を1つの文字列に結合する
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# RAGチェーンの構築
# プロンプト、LLM、文字列出力パーサーを組み合わせてチェーンを作成
rag_chain = prompt | llm | StrOutputParser()

# チェーンの実行
# コンテキスト（検索された文書）と質問を入力として与え、生成を実行
context = format_docs(docs)
question_ja = "エージェントのメモリについて教えてください"  # 日本語の質問例
generation = rag_chain.invoke({"context": context, "question": question_ja})

# 生成結果の表示
print(generation)

エージェントのメモリは、長期記憶モジュール（外部データベース）であり、自然言語でエージェントの経験の包括的なリストを記録します。このメモリは、過去の経験に基づいてエージェントが行動し、他のエージェントとの相互作用を可能にするためにLLM（大規模言語モデル）と組み合わされています。


In [21]:
### 質問リライター（日本語版）

# 必要なライブラリをインポート
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# LLMの設定
# OpenAIのGPT-3.5-turboモデルを使用し、temperature=0で決定論的な出力を得る
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

# プロンプトの設定
system = """あなたは質問リライターです。入力された質問をウェブ検索に最適化されたより良いバージョンに変換します。
入力を見て、その背後にある意味的な意図や意味について推論してください。"""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "以下が元の質問です：\n\n {question} \n より良い質問を作成してください。",
        ),
    ]
)

# 質問リライターチェーンの構築
# プロンプト、LLM、文字列出力パーサーを組み合わせてチェーンを作成
question_rewriter = re_write_prompt | llm | StrOutputParser()

# テスト実行
# サンプルの質問を用意し、リライターを実行
sample_question = "AIエージェントの記憶について教えて"
improved_question = question_rewriter.invoke({"question": sample_question})

# 結果の表示
print(f"元の質問: {sample_question}")
print(f"改善された質問: {improved_question}")

元の質問: AIエージェントの記憶について教えて
改善された質問: AIエージェントが情報をどのように記憶し、それを活用するのかについて詳しく教えてください。


#**Web Search Tool**

In [22]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)

#**Graph**

##**Graph State**

In [23]:
from typing import List
from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    グラフの状態を表現するクラス。

    属性:
        question: 質問
        generation: LLMによる生成結果
        web_search: Web検索を追加するかどうか
        documents: 文書のリスト
    """

    question: str  # 質問（文字列）
    generation: str  # LLMによる生成結果（文字列）
    web_search: str  # Web検索を追加するかどうか（'Yes'または'No'を想定）
    documents: List[str]  # 文書のリスト（文字列のリスト）

In [24]:
from langchain.schema import Document

def retrieve(state):
    """
    文書を検索する

    引数:
        state (dict): 現在のグラフの状態

    戻り値:
        state (dict): 検索された文書を含む新しいキー 'documents' が追加された状態
    """
    print("---文書検索---")
    question = state["question"]

    # 検索の実行
    documents = retriever.get_relevant_documents(question)
    return {"documents": documents, "question": question}

def generate(state):
    """
    回答を生成する

    引数:
        state (dict): 現在のグラフの状態

    戻り値:
        state (dict): LLMによる生成結果を含む新しいキー 'generation' が追加された状態
    """
    print("---回答生成---")
    question = state["question"]
    documents = state["documents"]

    # RAG生成
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

def grade_documents(state):
    """
    検索された文書が質問に関連しているかどうかを判断する

    引数:
        state (dict): 現在のグラフの状態

    戻り値:
        state (dict): 関連性のある文書のみでフィルタリングされた 'documents' キーを含む状態
    """
    print("---文書の関連性チェック---")
    question = state["question"]
    documents = state["documents"]

    # 各文書のスコアリング
    filtered_docs = []
    web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---評価: 文書は関連あり---")
            filtered_docs.append(d)
        else:
            print("---評価: 文書は関連なし---")
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question": question, "web_search": web_search}

def transform_query(state):
    """
    より良い質問を生成するためにクエリを変換する

    引数:
        state (dict): 現在のグラフの状態

    戻り値:
        state (dict): 言い換えられた質問で更新された 'question' キーを含む状態
    """
    print("---クエリ変換---")
    question = state["question"]
    documents = state["documents"]

    # 質問の書き直し
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}

def web_search(state):
    """
    言い換えられた質問に基づいてWeb検索を実行する

    引数:
        state (dict): 現在のグラフの状態

    戻り値:
        state (dict): Web検索結果が追加された 'documents' キーを含む状態
    """
    print("---Web検索---")
    question = state["question"]
    documents = state["documents"]

    # Web検索の実行
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    documents.append(web_results)

    return {"documents": documents, "question": question}

### エッジ

def decide_to_generate(state):
    """
    回答を生成するか、質問を再生成するかを決定する

    引数:
        state (dict): 現在のグラフの状態

    戻り値:
        str: 次に呼び出すノードの二者択一の決定
    """
    print("---評価済み文書の判定---")
    state["question"]
    web_search = state["web_search"]
    state["documents"]

    if web_search == "Yes":
        # すべての文書が関連性チェックでフィルタリングされた
        # 新しいクエリを再生成する
        print(
            "---決定: すべての文書が質問に関連していないため、クエリを変換---"
        )
        return "transform_query"
    else:
        # 関連する文書があるため、回答を生成
        print("---決定: 回答生成---")
        return "generate"

#**Build Graph**

In [25]:
from langgraph.graph import END, StateGraph, START

# GraphStateを使用してStateGraphを初期化
workflow = StateGraph(GraphState)

# ノードの定義
workflow.add_node("retrieve", retrieve)  # 文書検索
workflow.add_node("grade_documents", grade_documents)  # 文書評価
workflow.add_node("generate", generate)  # 回答生成
workflow.add_node("transform_query", transform_query)  # クエリ変換
workflow.add_node("web_search_node", web_search)  # Web検索

# グラフの構築
workflow.add_edge(START, "retrieve")  # 開始から文書検索へ
workflow.add_edge("retrieve", "grade_documents")  # 文書検索から文書評価へ

# 条件付きエッジの追加：文書評価の結果に基づいて次のステップを決定
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",  # クエリ変換が必要な場合
        "generate": "generate",  # 回答生成が可能な場合
    },
)

workflow.add_edge("transform_query", "web_search_node")  # クエリ変換からWeb検索へ
workflow.add_edge("web_search_node", "generate")  # Web検索から回答生成へ
workflow.add_edge("generate", END)  # 回答生成から終了へ

# グラフのコンパイル
app = workflow.compile()

# 使用例：
# inputs = {"question": "AIの倫理について教えてください"}
# for output in app.stream(inputs):
#     for key, value in output.items():
#         print(f"ノード '{key}'の出力:")
#         print(value)
#     print("\n---\n")

In [26]:
from pprint import pprint

# 実行
inputs = {"question": "エージェントのメモリの種類は何ですか？"}
for output in app.stream(inputs):
    for key, value in output.items():
        # ノード名の表示
        pprint(f"ノード '{key}':")
        # オプション: 各ノードでの完全な状態を表示
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# 最終的な生成結果の表示
pprint(value["generation"])

# 注: このコードを実行すると、ワークフローの各ステップごとに出力が表示されます。
# 最後に生成された回答が表示されます。

---文書検索---
"ノード 'retrieve':"
'\n---\n'
---文書の関連性チェック---
---評価: 文書は関連あり---
---評価: 文書は関連あり---
---評価: 文書は関連あり---
---評価: 文書は関連あり---
---評価済み文書の判定---
---決定: 回答生成---
"ノード 'grade_documents':"
'\n---\n'
---回答生成---
"ノード 'generate':"
'\n---\n'
('エージェントのメモリの種類には、次のようなものがあります：\n'
 '- 感覚記憶（Sensory Memory）: '
 '視覚的、聴覚的な情報などの印象を保持し、元の刺激が終了した後も情報を保持する能力を提供します。通常、数秒間しか続かないことが一般的です。視覚的なものをアイコニックメモリ（iconic '
 'memory）、聴覚的なものをエコーイックメモリ（echoic memory）、触覚的なものをハプティックメモリ（haptic '
 'memory）というサブカテゴリに分類されます。')


In [27]:
from pprint import pprint

# 実行
inputs = {"question": "AlphaCodium論文はどのように機能しますか？"}
for output in app.stream(inputs):
    for key, value in output.items():
        # ノード名の表示
        pprint(f"ノード '{key}':")
        # オプション: 各ノードでの完全な状態を表示
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# 最終的な生成結果の表示
pprint(value["generation"])

---文書検索---
"ノード 'retrieve':"
'\n---\n'
---文書の関連性チェック---
---評価: 文書は関連なし---
---評価: 文書は関連なし---
---評価: 文書は関連なし---
---評価: 文書は関連なし---
---評価済み文書の判定---
---決定: すべての文書が質問に関連していないため、クエリを変換---
"ノード 'grade_documents':"
'\n---\n'
---クエリ変換---
"ノード 'transform_query':"
'\n---\n'
---Web検索---
"ノード 'web_search_node':"
'\n---\n'
---回答生成---
"ノード 'generate':"
'\n---\n'
('AlphaCodiumは、大規模言語モデル（LLM）を使用してコード生成性能を向上させるための新しいコード生成プロセスです。主な特徴や機能は以下の通りです：\n'
 '\n'
 '1. GPT-4のコード生成精度を2倍以上向上させることができる。\n'
 '2. 生成されたコードを繰り返し実行し、入出力テストに対して修正する反復プロセスを中心にしている。\n'
 '3. AlphaCodiumは、競技プログラミングタスクに特化しており、多くの可能な解決策を生成し、その中から最適なものを選ぶ手法を採用している。\n'
 '4. 計算コストが比較的少なく、実用的な性能を持っている。\n'
 '5. '
 'AlphaCodiumは、AlphaCodeと比較してより優れており、はるかに少ない計算資源を使用しながらより良い結果を出していることが示されている。\n'
 '\n'
 'これらの特徴により、AlphaCodiumはコード生成の精度と効率を向上させるための有望な手法として注目されています。')
